### Import Essential Libraries

In [150]:
import pandas as pd
import numpy as np

### Import Data Visualization Libraries

In [151]:
import matplotlib.pyplot as plt 
import seaborn as sns 
import altair as alt 

### Import Preprocessing Packages

#### To create training and testing datasets 

In [152]:
from sklearn.model_selection import train_test_split 

### To impute the data

In [153]:
from sklearn.impute import SimpleImputer, KNNImputer

### To standarize and scale the data correctly 

In [154]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder 

### To create the pipeline

In [155]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

#### To Evaluate Quality of Logistic Regression Model

In [156]:
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import (
    confusion_matrix,
    precision_score,
    recall_score,
    roc_curve,
    roc_auc_score
)
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.metrics import classification_report

#### Create a random seed for reproducability

In [157]:
random_seed = 172193
np.random.seed(random_seed)

---

#### Load in the dataset

In [158]:
df = pd.read_excel('Employee_Data_Project.xlsx')
print(f'The Dataset shape is: {df.shape}')

The Dataset shape is: (4410, 18)


#### Look at the data structure

In [159]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      4410 non-null   int64  
 1   Attrition                4410 non-null   object 
 2   BusinessTravel           4410 non-null   object 
 3   DistanceFromHome         4410 non-null   int64  
 4   Education                4410 non-null   int64  
 5   EmployeeID               4410 non-null   int64  
 6   Gender                   4410 non-null   object 
 7   JobLevel                 4410 non-null   int64  
 8   MaritalStatus            4410 non-null   object 
 9   Income                   4410 non-null   int64  
 10  NumCompaniesWorked       4391 non-null   float64
 11  StandardHours            4410 non-null   int64  
 12  TotalWorkingYears        4401 non-null   float64
 13  TrainingTimesLastYear    4410 non-null   int64  
 14  YearsAtCompany          

---

#### Check the rows with nulls 

In [160]:
missing = df.isna().sum()
## to get the percentage missing (divide by the length of the dataframe)
missing_per = (missing / len(df)) *100

print(missing)
print(missing_per)

Age                         0
Attrition                   0
BusinessTravel              0
DistanceFromHome            0
Education                   0
EmployeeID                  0
Gender                      0
JobLevel                    0
MaritalStatus               0
Income                      0
NumCompaniesWorked         19
StandardHours               0
TotalWorkingYears           9
TrainingTimesLastYear       0
YearsAtCompany              0
YearsWithCurrManager        0
EnvironmentSatisfaction    25
JobSatisfaction            20
dtype: int64
Age                        0.000000
Attrition                  0.000000
BusinessTravel             0.000000
DistanceFromHome           0.000000
Education                  0.000000
EmployeeID                 0.000000
Gender                     0.000000
JobLevel                   0.000000
MaritalStatus              0.000000
Income                     0.000000
NumCompaniesWorked         0.430839
StandardHours              0.000000
TotalWorkingYea

---

In [161]:
df['StandardHours'].nunique()


1

#### Since standard hours is all constant, will drop this column

---

In [162]:
df.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DistanceFromHome', 'Education',
       'EmployeeID', 'Gender', 'JobLevel', 'MaritalStatus', 'Income',
       'NumCompaniesWorked', 'StandardHours', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'YearsAtCompany', 'YearsWithCurrManager',
       'EnvironmentSatisfaction', 'JobSatisfaction'],
      dtype='object')

In [163]:
target = 'Attrition'
drop = ['EmployeeID' , 'StandardHours']
num_vars = ['Age' , 'DistanceFromHome' , 'Income' , 'NumCompaniesWorked' , 'TotalWorkingYears' , 'TrainingTimesLastYear'
            , 'YearsAtCompany' , 'YearsWithCurrManager']
ordinal_vars = ['Education' , 'JobLevel' , 'EnvironmentSatisfaction' , 'JobSatisfaction']
nom_vars = ['Gender' , 'BusinessTravel' , 'MaritalStatus']


---

In [164]:
df['Attrition_lab'] = df['Attrition'].map({'Yes':1 , 'No' :0})


---

In [165]:
X = df[num_vars+ordinal_vars+nom_vars].copy()
y= df['Attrition_lab'].copy()

X_train , X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, stratify=y, random_state=random_seed)

---

#### Build Preprocessing Pipeline 


In [166]:
## need to transform the numeric columns, nominal and ordinal columns 
### KNN imputation for the numerical columns
numeric_imputation = Pipeline(steps = [(
    'imputer' , KNNImputer(n_neighbors=5 , weights='uniform')),
('scaler' , StandardScaler())])
### regarding the nominal variables , use onehotencoding and replace the nulls with the mode (most frequent)
nominal_imputation = Pipeline(steps=[(
'imputer' , SimpleImputer(strategy= 'most_frequent')),
('one_hot_encoder' , OneHotEncoder(handle_unknown= 'ignore' , sparse_output= False , drop = 'first'))])
### regarding the ordinal variables replace with the median 
ordinal_imputation = Pipeline(steps= [('imputer' , SimpleImputer(strategy='median'))])

In [167]:
pre_process = ColumnTransformer(transformers=[('numeric' , numeric_imputation , num_vars),
                                              ('ordinal' , ordinal_imputation, ordinal_vars),
                                              ('nominal' , nominal_imputation , nom_vars)])


In [168]:
X_train_processed = pre_process.fit_transform(X_train)  
X_test_processed = pre_process.transform(X_test)


In [169]:
feature_names = pre_process.get_feature_names_out()
feature_names

array(['numeric__Age', 'numeric__DistanceFromHome', 'numeric__Income',
       'numeric__NumCompaniesWorked', 'numeric__TotalWorkingYears',
       'numeric__TrainingTimesLastYear', 'numeric__YearsAtCompany',
       'numeric__YearsWithCurrManager', 'ordinal__Education',
       'ordinal__JobLevel', 'ordinal__EnvironmentSatisfaction',
       'ordinal__JobSatisfaction', 'nominal__Gender_Male',
       'nominal__BusinessTravel_Travel_Frequently',
       'nominal__BusinessTravel_Travel_Rarely',
       'nominal__MaritalStatus_Married', 'nominal__MaritalStatus_Single'],
      dtype=object)

In [170]:
X_train_processed_df = pd.DataFrame(X_train_processed, columns=feature_names)
X_test_processed_df = pd.DataFrame(X_test_processed, columns= feature_names)

X_train_processed_df

,numeric__Age,numeric__DistanceFromHome,numeric__Income,numeric__NumCompaniesWorked,numeric__TotalWorkingYears,numeric__TrainingTimesLastYear,numeric__YearsAtCompany,numeric__YearsWithCurrManager,ordinal__Education,ordinal__JobLevel,ordinal__EnvironmentSatisfaction,ordinal__JobSatisfaction,nominal__Gender_Male,nominal__BusinessTravel_Travel_Frequently,nominal__BusinessTravel_Travel_Rarely,nominal__MaritalStatus_Married,nominal__MaritalStatus_Single
0,-0.204878,-0.155259,-0.574737,-0.666602,-1.304028,-0.601912,-0.980175,-0.866629,4.0,3.0,4.0,4.0,0.0,0.0,1.0,1.0,0.0
1,1.009692,-0.892809,2.148524,2.148666,2.151124,0.960832,3.110955,1.365332,3.0,2.0,3.0,1.0,1.0,0.0,1.0,1.0,0.0
2,0.126368,1.319841,-0.962122,0.137760,1.127376,0.179460,1.801793,1.923322,4.0,2.0,2.0,3.0,0.0,0.0,1.0,1.0,0.0
3,0.788861,2.426165,-0.978826,0.539942,-0.152310,0.960832,-0.816530,-0.587634,4.0,2.0,1.0,3.0,1.0,0.0,1.0,0.0,1.0
4,-2.081942,-1.015734,-0.896595,-0.666602,-1.431996,0.179460,-1.143820,-1.145624,4.0,1.0,3.0,3.0,1.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3082,-0.425709,0.582291,0.920412,0.942123,-0.024342,-0.601912,0.328987,0.807342,3.0,1.0,4.0,2.0,1.0,0.0,0.0,0.0,0.0
3083,0.126368,0.459366,-0.709219,-1.068783,-1.048091,1.742204,-0.816530,-0.587634,3.0,1.0,3.0,3.0,1.0,1.0,0.0,1.0,0.0
3084,1.230523,-0.401109,0.927693,-0.264421,0.359564,2.523576,0.328987,0.807342,3.0,1.0,2.0,4.0,1.0,0.0,1.0,1.0,0.0
3085,0.568030,1.688615,-0.830853,-1.068783,1.255344,0.179460,2.129084,1.365332,3.0,3.0,3.0,1.0,0.0,0.0,1.0,1.0,0.0


#### Verify if there are now any nulls in the processed dataset

In [171]:
print(X_train_processed_df.isna().sum())
print(X_test_processed_df.isna().sum())

numeric__Age                                 0
numeric__DistanceFromHome                    0
numeric__Income                              0
numeric__NumCompaniesWorked                  0
numeric__TotalWorkingYears                   0
numeric__TrainingTimesLastYear               0
numeric__YearsAtCompany                      0
numeric__YearsWithCurrManager                0
ordinal__Education                           0
ordinal__JobLevel                            0
ordinal__EnvironmentSatisfaction             0
ordinal__JobSatisfaction                     0
nominal__Gender_Male                         0
nominal__BusinessTravel_Travel_Frequently    0
nominal__BusinessTravel_Travel_Rarely        0
nominal__MaritalStatus_Married               0
nominal__MaritalStatus_Single                0
dtype: int64
numeric__Age                                 0
numeric__DistanceFromHome                    0
numeric__Income                              0
numeric__NumCompaniesWorked                  0


#### Confirmed that now there are no NAs

----

#### Downsample

##### During the Preprocessing Pipeline, was converted to a np.array 
##### Convert to np.array to match X_train_processed

In [182]:
y_train_array = y_train.values
pos_idx = np.where(y_train_array == 1)[0]
neg_idx = np.where(y_train_array == 0 )[0]

#### Set randomseed again 

In [183]:
np.random.seed(random_seed)
keep_neg= np.random.choice(neg_idx, size = len(pos_idx), replace = False)
keep_idx = np.concatenate([pos_idx, keep_neg])

y_train_balanced = y_train_array[keep_idx]
X_train_balanced = X_train_processed[keep_idx]

In [184]:
y_train_balanced_df = pd.DataFrame(y_train_balanced , columns= ['Attrition_lab'])
X_train_processed_balanced_df = pd.DataFrame(X_train_balanced , columns = feature_names)

---

# Output: 
- X_train_processed_balanced_df (downampled and preprocessed with KNN+simple imputer)
- X_train_processed_df (preprocessed not downsampled)
- y_train_balanced (use with X_train_processsed_balanced_df)
- y_train (use with X_train_processed_df)
- X_test_processed_df (preprocessed with KNN+simple imputer pipeline)
- y_test (use with X_test_processed)

- y_train_balanced_df can be used however will need to do y_train_balanced_df_values.ravel() (this is since model fit requires 1d array)



---

#### Test to see if this works with logistic regression model

In [185]:
log_reg = LogisticRegression(
    solver="lbfgs",
    max_iter=1000
)

log_reg.fit(X_train_processed_balanced_df, y_train_balanced_df.values.ravel())


LogisticRegression(max_iter=1000)

#### Predict and eval on test set

In [186]:
y_pred = log_reg.predict(X_test_processed_df)

print("Test AUC (median imputation + standardization):", roc_auc_score(y_test, y_pred))

Test AUC (median imputation + standardization): 0.67638624540033


#### Initiate logistic regression model

In [187]:
log_reg = LogisticRegression(
    solver="lbfgs",
    max_iter=1000
)

#### Fit logistic regression model

In [188]:
log_reg.fit(X_train_processed_balanced_df, y_train_balanced)

LogisticRegression(max_iter=1000)

### Make Predictions

#### Predict and eval on test set

In [189]:
y_pred = log_reg.predict(X_test_processed_df)

#### Print Test AUC

In [190]:
print("Test AUC (median imputation + standardization):", roc_auc_score(y_test, y_pred))

Test AUC (median imputation + standardization): 0.67638624540033


#### Probabilities

In [191]:
y_prob = log_reg.predict_proba(X_test_processed_df)
print(y_prob)

[[0.55942533 0.44057467]
 [0.62255083 0.37744917]
 [0.44778131 0.55221869]
 ...
 [0.3694525  0.6305475 ]
 [0.85662624 0.14337376]
 [0.34968883 0.65031117]]


#### Evaluate Logistic Regression Model

In [192]:
print(confusion_matrix(y_test, y_pred))

[[772 338]
 [ 73 140]]


In [193]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.70      0.79      1110
           1       0.29      0.66      0.41       213

    accuracy                           0.69      1323
   macro avg       0.60      0.68      0.60      1323
weighted avg       0.81      0.69      0.73      1323



#### Inspect Coefficients

In [196]:
coefficients = pd.DataFrame({
    "feature": X_test_processed_df.columns,
    "coefficient": log_reg.coef_[0]
})

coefficients

,feature,coefficient
0,numeric__Age,-0.188005
1,numeric__DistanceFromHome,0.047046
2,numeric__Income,-0.057745
3,numeric__NumCompaniesWorked,0.150209
4,numeric__TotalWorkingYears,-0.451246
5,numeric__TrainingTimesLastYear,-0.133343
6,numeric__YearsAtCompany,0.441717
7,numeric__YearsWithCurrManager,-0.611985
8,ordinal__Education,-0.116350
9,ordinal__JobLevel,-0.013396
